# CoCoRaHS AI - Intro to Topic Modeling

The goal here is to do exploratory data analysis (EDA) on the observation notes data available in the CoCoRaHS data source. For this analysis we looked at the DailyPrecipReport that has around **46.5 million** records, out of which around **8 million** records have observation notes. This study takes a random 1% sample from the 8 million records (**91,937** records with notes) to perform the initial EDA. 

Since we are dealing with large amount of unstructured and unlabeled data, the EDA is going to leverage two key unsupervised machine learning algorithms that are suitable for text analysis. Specifically, we will be looking at two topic modeling algorithms: **Latent Dirichlet Allocation (LDA)** and **K-Means Clustering**. 

This notebook is divided into the following sections:

- Setup – load and review the data from notes
- Breakdown of weather terms in the notes
- Topic Modeling with LDA
- Topic Modeling with K-Means Clustering
- Compare predictions from LDA and K-Means on test data

## Setup

**Import required libraries**

In [1]:
import pandas as pd
import numpy as np
import sklearn
from IPython.display import display, HTML, Image
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', -1)
print("pandas version: {} numpy version: {} sklearn version: {}".format(pd.__version__, 
                                                                        np.__version__, sklearn.__version__))

pandas version: 0.24.2 numpy version: 1.16.4 sklearn version: 0.21.2


In [2]:
import nltk
nltk.__version__

'3.4.4'

**Load the sample 1% data with observation notes from DailyPrecipReport table**

Note that data was sampled in SQL and the correspoing csv file was saved in Azure blob storage. Here is SQL that was used to sample the data.

``` mysql
select [Notes] from [dbo].[DailyPrecipReport] where [DailyPrecipReportID] in 
(
    select top 1 percent [DailyPrecipReportID] 
    from [dbo].[DailyPrecipReport] 
    WHERE 
    [Notes] IS NOT NULL
    AND
    TRIM([Notes])  <> ''
    order by newid()
)
```

In [2]:
data_url = ('https://cocorahsai.blob.core.windows.net/notes/percent_1.csv')
df = pd.read_csv(data_url, names=['notes'], sep=u'\0091')
df['notes'] = df['notes'].astype('str')
print(len(df))

91937


Helper method to display dataframes

In [16]:
def display_dataframe(df_in):
    s = df_in.style.set_properties(**{'text-align': 'left'})
    s.set_table_styles([dict(selector='th', props=[('text-align', 'left')])])
    display(HTML(s.render()))

In [15]:
# print the first 10 rows
display_dataframe(df.head(10))

,notes
0,Very intense lightning from approx. 8:30 to 10:00
1,"Total snow = 0, new snow = 0"
2,Rain yesterday AM --snow in parts of town
3,"5/8 @ 12:35am 0.02""5/8 @ 4:15pm 0.11""5/8 @ 7:30pm 0.10""5/9 @ 7:00am 0.01"""
4,bulk data:
5,"HAIL MIXED WITH RAIN FROM 12:45 TO 1 P.M. ON JULY 1. DIAMETER ~3/8 INCH. HAIL PAD TURNED IN OUTSIDE YOUR OFFICE EARLY THIS MORNING (7/3). I WILL BE OUT OF TOWN FROM 7/6 UNTIL 7/23 AND UNABLE TO MAKE PRECIPITATION REPORTS. ALSO, PLEASE SEE WHAT YOU CAN DO WITH YOUR EMAIL SYSTEM. I HAVE RECEIVED AT LEAST A DOZEN EMAILS FROM PEOPLE WHO RECEIVED ERRONEOUS EMAIL TELLING YOU TO REMOVE THEM FROM YOUR LIST. THAT IS MORE ANNOYING THAT GETTING A SINGLE ERRONEOUS MESSAGE! (I KNOW WHY IT IS HAPPENING, I READ THAT EMAIL, TOO. PERHAPS CHANGE YOUR INTERNAL ADDRESS?) REGARDS, GEORGEANN"
6,bulk data:
7,"snow was falling at 7am 1/16/01 and continued until some time during night of 1/17/01.1/16/01 1:00pm total 4.6"" snow 1.3"" new snow1/16/01 1:00pm total 0.28"" moisture 0.06"" new moisture1/16/01 6:30pm total 5.0"" snow 0.4"" new snow1/16/01 6:30pm total 0.3"" moisture 0.02"" new moisture"
8,Total percipitaion was a combination of rain and snow.
9,Data from city automatic station.


The data shows both very short and very long notes. Let's look at the stats on the length (word counts) of the notes.

In [5]:
notes_len = df.notes.map(lambda x: len(x))
notes_len.describe()

count    91937.000000
mean     61.335273   
std      75.150907   
min      1.000000    
25%      25.000000   
50%      39.000000   
75%      71.000000   
max      1976.000000 
Name: notes, dtype: float64

## Breakdown of Weather Terms

We start with defining a list of weather events of interested. Then we will look at the break down of the these weather events or terms as mentioned in the notes.

In [6]:
# Create weather events dictionary
weather_events = []
weather_events.append('rain')
weather_events.append('snow')
weather_events.append('precipitation')
weather_events.append('wind')
weather_events.append('fog')
weather_events.append('hail')
weather_events.append('sun')
weather_events.append('cloud')
weather_events.append('thunder')
weather_events.append('lightning')
weather_events.append('warm')
weather_events.append('hot')
weather_events.append('cold')
weather_events.append('freeze')
weather_events.append('arid')
weather_events.append('humid')
weather_events.append('thunderstorms')
weather_events.append('downbursts')
weather_events.append('tornadoes')
weather_events.append('cyclones')
weather_events.append('waterspouts')
weather_events.append('flood')
weather_events.append('blizzards')
weather_events.append('snowstorms')
weather_events.append('ice storms')
weather_events.append('dust storms')

In [25]:
# Make a copy of the originial dataframe
df_2 = df.copy()
df_2['notes'] = df_2['notes'].astype('str')

# Search of the weather events or terms in the notes
evt_counts = []
for evt in weather_events:
    df_2[evt] = df_2['notes'].apply(lambda x: 1 if (evt in x) else 0)
    count = len(df_2.where(df_2[evt] == 1).dropna())
    percent = round(count/len(df_2) * 100, 1)
    evt_counts.append((evt, count, percent))

# Create a new weather events dataframe
evt_df = pd.DataFrame(evt_counts, columns=['Event', 'Count', 'Percent'])
evt_df.sort_values(by=['Count'], ascending=False, inplace=True)

display_dataframe(evt_df)

,Event,Count,Percent
0,rain,11744,12.8
3,wind,9239,10
7,cloud,8646,9.4
1,snow,7220,7.9
6,sun,3895,4.2
4,fog,2469,2.7
15,humid,2263,2.5
8,thunder,1850,2
10,warm,1293,1.4
12,cold,1274,1.4


#### Breakdown of Top Weather Terms (>= 2%)

![Breakdown of Top Weather Terms (>= 2%)](./images/w_dist.svg 'Breakdown of Top Weather Terms (>= 2%)')

## Topic Modeling with LDA

**What is topic modeling?**

- It is a high-level analysis of what's in a text collection
- Topic is a theme or subject of the discussion
- Topics are represented as word distributions
- A single document can be a mixture of multiple topics


**What is LDA?**

LDA is a generative probabilistic model for a collection of text corpora (large and structured set of texts). In LDA, each item (or word) of a collection is modeled as a finite mixture over an underlying set of topics. Each topic is, in turn modeled as an infinite mixture over an underlying set of topic probabilities. Words are generated by topics (by fixed conditional distribution) and topics are infinitely exchangeable within a document.


**Practical implications of LDA**

- How many topics?
  - Finding or even guessing the number of topics is hard
- Interpreting topics
  - Topics are just word distributions
- Making sense of words / generating labels is subjective

**Data Cleaning**

Remove autogenerated notes that does not have qualitative weather information

In [19]:
# define list of autogenerated notes
words = ['MSWS data', 'Data from city automatic station', 'bulk data', 'LRM Estimates',
        'Observer for iOS', 'Observer for Android']

# remove documents with autogenerated notes
df_3 = df[~np.logical_or.reduce([df['notes'].str.contains(word, case = False) for word in words])]

print("review impact of removing autogenerated notes")
print('original size: {} new size: {} difference: {}'.format(len(df), len(df_3), len(df) - len(df_3)))
print()

print("stats on the length of the notes")
notes_len = df_3.notes.map(lambda x: len(x))
notes_len.describe()

review impact of removing autogenerated notes
original size: 91937 new size: 71636 difference: 20301

stats on the length of the notes


count    71636.000000
mean     65.862611   
std      84.081556   
min      1.000000    
25%      21.000000   
50%      41.000000   
75%      81.000000   
max      1976.000000 
Name: notes, dtype: float64

### Generate Corpus

Generate a vocabulary or collection of words that will be used in topic modeling. As a standard practice, we will filter out all stop words in English (for e.g. words like THE, THIS, BUT etc). The two other key parameters are `min_df`, and `max_df`. In this case, we have chosen **min_df = 100**, that means we will consider only those words that appear in atleast 100 different documents or notes. We have chosen **max_df = 20%**, that means we will ignore words that appear in more that 20% of the documents or notes. Note that the most frequent word **rain** was present in **11,744** documents, that is around **16%** of the new document count of **71,636**, thus we have set `max_df` to be at 20%.

In [20]:
import gensim
from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer(min_df=100, max_df=0.2, stop_words='english', 
                       token_pattern='\\b[a-zA-Z][a-zA-Z][a-zA-Z]+\\b')
X = vect.fit_transform(df_3.notes.values)
id_map = dict((v, k) for k, v in vect.vocabulary_.items())
corpus = gensim.matutils.Sparse2Corpus(X, documents_columns=False)

print('vocabulary size: {}'.format(len(vect.vocabulary_.items())))

vocabulary size: 487


### Generate the LDA model

For the current iteration we have kept the number of topics to 5 and this will also simplify interpretability and label generation. In the future, we should experiment to find optimal number of topics for the current corpus that balances granularity and interpretability, and thus making it both practical and useable.

In [21]:
num_topics = 5
ldamodel = gensim.models.ldamodel.LdaModel(corpus=corpus, num_topics=num_topics, id2word=id_map, passes=25, 
                                          random_state=31)

### Review Generated Topics

![Review Generated Topics](./images/word_cloud.svg 'Review Generated Topics')

In [22]:
cols = [np.hstack((['Topic 1: Precipitation'] * 2, ['Topic 2: Temperature'] * 2, 
                  ['Topic 3: Sky Condition'] * 2, ['Topic 4: General Conditions'] * 2, ['Topic 5: Time'] * 2)), 
        ['Word', 'Probability'] * 5]

topics = np.hstack((ldamodel.show_topic(0), ldamodel.show_topic(1), ldamodel.show_topic(2), 
                   ldamodel.show_topic(3), ldamodel.show_topic(4)))

results = pd.DataFrame(topics, index=[i+1 for i in range(10)], columns = cols)

results

Topic 1: Precipitation              Topic 2: Temperature               \
                     Word  Probability                 Word  Probability   
1   rain                   0.13208745   low                  0.16830178    
2   snow                   0.08925175   high                 0.16747908    
3   light                  0.05273157   temp                 0.12913682    
4   night                  0.021623608  max                  0.038666878   
5   yesterday              0.020393576  cold                 0.033644367   
6   morning                0.017624294  min                  0.031264827   
7   heavy                  0.017107379  overnight            0.02601982    
8   gauge                  0.01686538   hum                  0.02175416    
9   time                   0.014078558  temps                0.02106582    
10  overnight              0.013556986  hrs                  0.01761639    

   Topic 3: Sky Condition              Topic 4: General Conditions  \
                     Word  Probability                        Word   
1   clear                  0.15069275   wind                         
2   degrees                0.13019721   cloudy                       
3   calm                   0.053928424  overcast                     
4   fog                    0.047822762  mph                          
5   light                  0.045827277  temp                         
6   sunny                  0.038536187  partly                       
7   observation            0.035941005  humidity                     
8   skies                  0.034389354  calm                         
9   temperature            0.033172343  dew                          
10  cloudy                 0.031655487  winds                        

                Topic 5: Time               
    Probability          Word  Probability  
1   0.12137897   yesterday     0.057131898  
2   0.09703539   morning       0.054491155  
3   0.05775906   day           0.052992664  
4   0.05578674   afternoon     0.045030233  
5   0.048260946  today         0.025589414  
6   0.039538704  clouds        0.025227925  
7   0.03848893   sunny         0.02459881   
8   0.03334488   late          0.022544773  
9   0.027130133  report        0.018829342  
10  0.026088884  warm          0.01872901

### Topic Distribution

In LDA, each document may be viewed as a mixture of various topics where each document is considered to have a set of topics that are assigned to it via LDA. To get an idea of topic distribution across the entire corpus, we will assign each document to a topic based on the topic probability for that document – we will pick the topic with highest probability for that document.

In [23]:
topic_names = ['Topic 1: Precipitation', 'Topic 2: Temperature', 'Topic 3: Sky Condition', 
             'Topic 4: General Conditions', 'Topic 5: Time']

document_topics = ldamodel.get_document_topics(corpus)
doc_topic_list = []
doc_topic_prob_list = []
for doc_topic in document_topics:
    t_group, t_group_prob = (max(doc_topic, key=lambda item:item[1]))
    doc_topic_list.append(t_group)
    doc_topic_prob_list.append(t_group_prob)

_, counts = np.unique(doc_topic_list, return_counts=True)
topic_dist = []
for i in range(len(counts)):
    topic_dist.append((i+1, topic_names[i], counts[i], round(counts[i]/len(document_topics) * 100, 1)))

topic_dist_pd = pd.DataFrame(topic_dist, columns=['Topic #', 'Topic Name', 'Counts', 'Percent'])
display_dataframe(topic_dist_pd)

,Topic #,Topic Name,Counts,Percent
0,1,Topic 1: Precipitation,25083,35
1,2,Topic 2: Temperature,9172,12.8
2,3,Topic 3: Sky Condition,13555,18.9
3,4,Topic 4: General Conditions,14325,20
4,5,Topic 5: Time,9501,13.3


![Topic Distribution LDA](./images/td_lda.svg 'Topic Distribution LDA')

## Topic Modeling with K-Means Clustering

**What is K-Means Clustering?**

K-Means Clustering is a discriminative, unsupervised cluster analysis algorithm that aims to partition n observations (documents) into k disjoint clusters (topics). For a given number of clusters, the goal of the algorithm is to minimize the square distance between an observation and the assigned cluster centroid by varying both the assigned cluster and the cluster centroid. Unlike LDA where each observation can belong to multiple clusters, K-Means is considered hard clustering as each observation is assigned to one and only one cluster.

### Generate the Clusters / Topics

We will use the same hyperparameters as the LDA model: **min_df: 100**, **max_df: 0.2**, and **number of topics: 5**

In [26]:
import string
import collections
 
from nltk import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from pprint import pprint
 
 
def process_text(text, stem=True):
    text = text.translate(str.maketrans('','',string.punctuation))
    tokens = word_tokenize(text)
 
    if stem:
        stemmer = PorterStemmer()
        tokens = [stemmer.stem(t) for t in tokens]
 
    return tokens

vectorizer = TfidfVectorizer(tokenizer=process_text,
                             stop_words=stopwords.words('english'),
                             max_df=0.2,
                             min_df=100,
                             lowercase=True)

tfidf_model = vectorizer.fit_transform(df_3.notes.values)

n_clusters = 5
km_model = KMeans(n_clusters=n_clusters, random_state=95)
km_model.fit(tfidf_model)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=5, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=95, tol=0.0001, verbose=0)

### Review Generated Topics

K-Means assigns each document to a cluster. In order to interpret each cluster and assign labels, we will look at sample documents in the cluster to make the qualitative determination.

In [36]:
clusters = ['Topic 1: General', 'Topic 2: Cloudy', 'Topic 3: Temperature', 
            'Topic 4: Precipitation', 'Topic 5: Sky Condition']

# assign the predicted cluster for each document / note in the data frame
df_3['pred'] = km_model.labels_

topic_dist = []
for i in range(n_clusters):
    print('Topic #: {} Desc: {}'.format(i+1, clusters[i]))
    cdf = df_3[df_3.pred == i].notes
    topic_dist.append((i+1, clusters[i], len(cdf), round(len(cdf)*100.0/len(df_3), 1)))
    print(cdf.head(5))
    print()

Topic #: 1 Desc: Topic 1: General
0    Very intense lightning from approx. 8:30 to 10:00                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                
1    Total snow = 0, new snow = 0                                                                                                                                                                                                                                                                                                                                                              

### Topic Distribution

Since K-Means assigns each document to a cluster, we can very easily determine the overall cluster distribution for the corpus.

In [30]:
topic_dist_km_pd = pd.DataFrame(topic_dist, columns=['Topic #', 'Topic Name', 'Counts', 'Percent'])
display_dataframe(topic_dist_km_pd)

,Topic #,Topic Name,Counts,Percent
0,1,Topic 1: General,42970,60
1,2,Topic 2: Cloudy,5618,7.8
2,3,Topic 3: Temperature,8256,11.5
3,4,Topic 4: Precipitation,8874,12.4
4,5,Topic 5: Sky Condition,5918,8.3


In [ ]:
fig, ax = plt.subplots(figsize=(9, 5), subplot_kw=dict(aspect="equal"))

recipe = topic_dist_km_pd['Topic Name'].values

data = topic_dist_km_pd.Percent.values

#labels=list(map('{}%'.format, data))
wedges, texts = ax.pie(data, wedgeprops=dict(width=0.5), startangle=100, labels=list(map('{}%'.format, data)),
                       labeldistance=0.7, counterclock=False, textprops={'ha': 'center', 'fontsize': 8})

bbox_props = dict(boxstyle="square,pad=0.3", fc="w", ec="k", lw=0.72)
kw = dict(xycoords='data', textcoords='data', arrowprops=dict(arrowstyle="-"),
          bbox=bbox_props, zorder=0, va="center")

for i, p in enumerate(wedges):
    ang = (p.theta2 - p.theta1)/2. + p.theta1
    y = np.sin(np.deg2rad(ang))
    x = np.cos(np.deg2rad(ang))
    horizontalalignment = {-1: "right", 1: "left"}[int(np.sign(x))]
    connectionstyle = "angle,angleA=0,angleB={}".format(ang)
    kw["arrowprops"].update({"connectionstyle": connectionstyle})
    ax.annotate(recipe[i], xy=(x, y), xytext=(1.35*np.sign(x), 1.4*y),
                 horizontalalignment=horizontalalignment, **kw)
    
plt.title('Topic Distribution: KMeans Algorithm', y=1.03)

plt.show()

![Topic Distribution K-Means](./images/td_kmeans.svg 'Topic Distribution K-Means')

## Compare predictions from LDA and K-Means on test data

Both the LDA and K-Means model are used to automatically tag new incoming text data by using the topics learned. In this section, we will apply the learned models to previously unseen data (test data) to assign the new documents to their respective topics.

### Load and review test data

In [37]:
test_data_url = ('https://cocorahsai.blob.core.windows.net/notes/test/test_data.csv')
df_test = pd.read_csv(test_data_url, names=['notes'])
df_test['notes'] = df_test['notes'].astype('str')
display_dataframe(df_test)

,notes
0,It started raining about 8:45 in the morning and kept raining steadily. By 4:00 in the afternoon .61 of rain had fallen.
1,Current Conditions @ 0700: 32.5° Calm and clear to the west and partly cloudy to the east.
2,overnight low 28F
3,"Another day of heavy rain. Storm total of 2.50"""
4,"Clear, 79.3°; BP 30.10, rising."
5,there was hail with this storm. Very strong winds so don't know if all precip went in the guage.
6,frost and fog
7,"Overcast, 33."
8,"Current temp is 36F with light rain falling from cloudy skies and NW winds at 14 gusting to 26 mph. Yesterday's high temp was 66F (3 degrees short of the record of 69F) at 5:48 p.m. Light rain showers and drizzle fell off and on during the morning hours with light to moderate SE winds. By afternoon the showers ended and the winds shifted to light S with overcast skies. During the evening hours light SE winds prevailed with cloudy skies. Overnight, light rain showers began again with moderate NW winds gusting up to 26 mph. Temps dropped sharply during the night from the high 50s down to the mid 30s."
9,"Currently mostly cloudy, 30F."


### Assign Primary Topic to the test data using the LDA Model

In [39]:
X_test = vect.transform(df_test.notes.values)
test_corpus = gensim.matutils.Sparse2Corpus(X_test, documents_columns=False)
test_results = ldamodel[test_corpus]

topic_list = []
topic_prob_list = []
for test_topic in test_results:
    t_group, t_group_prob = (max(test_topic, key=lambda item:item[1]))
    topic_list.append(topic_names[t_group])
    topic_prob_list.append(t_group_prob)

df_test['Primary Topic'] = topic_list
df_test['Probability'] = topic_prob_list

display_dataframe(df_test)

,notes,Primary Topic,Probability
0,It started raining about 8:45 in the morning and kept raining steadily. By 4:00 in the afternoon .61 of rain had fallen.,Topic 1: Precipitation,0.674632
1,Current Conditions @ 0700: 32.5° Calm and clear to the west and partly cloudy to the east.,Topic 4: General Conditions,0.772648
2,overnight low 28F,Topic 2: Temperature,0.732454
3,"Another day of heavy rain. Storm total of 2.50""",Topic 1: Precipitation,0.865278
4,"Clear, 79.3°; BP 30.10, rising.",Topic 3: Sky Condition,0.400002
5,there was hail with this storm. Very strong winds so don't know if all precip went in the guage.,Topic 1: Precipitation,0.741296
6,frost and fog,Topic 3: Sky Condition,0.733333
7,"Overcast, 33.",Topic 4: General Conditions,0.6
8,"Current temp is 36F with light rain falling from cloudy skies and NW winds at 14 gusting to 26 mph. Yesterday's high temp was 66F (3 degrees short of the record of 69F) at 5:48 p.m. Light rain showers and drizzle fell off and on during the morning hours with light to moderate SE winds. By afternoon the showers ended and the winds shifted to light S with overcast skies. During the evening hours light SE winds prevailed with cloudy skies. Overnight, light rain showers began again with moderate NW winds gusting up to 26 mph. Temps dropped sharply during the night from the high 50s down to the mid 30s.",Topic 1: Precipitation,0.44198
9,"Currently mostly cloudy, 30F.",Topic 4: General Conditions,0.732796


### Predict Topic using the K-Means Model

In [40]:
pred = km_model.predict(vectorizer.transform(df_test.notes.values))
df_test['K-Means Topic'] = [clusters[i] for i in pred]

display_dataframe(df_test)

,notes,Primary Topic,Probability,K-Means Topic
0,It started raining about 8:45 in the morning and kept raining steadily. By 4:00 in the afternoon .61 of rain had fallen.,Topic 1: Precipitation,0.674632,Topic 4: Precipitation
1,Current Conditions @ 0700: 32.5° Calm and clear to the west and partly cloudy to the east.,Topic 4: General Conditions,0.772648,Topic 2: Cloudy
2,overnight low 28F,Topic 2: Temperature,0.732454,Topic 3: Temperature
3,"Another day of heavy rain. Storm total of 2.50""",Topic 1: Precipitation,0.865278,Topic 4: Precipitation
4,"Clear, 79.3°; BP 30.10, rising.",Topic 3: Sky Condition,0.400002,Topic 5: Sky Condition
5,there was hail with this storm. Very strong winds so don't know if all precip went in the guage.,Topic 1: Precipitation,0.741296,Topic 1: General
6,frost and fog,Topic 3: Sky Condition,0.733333,Topic 1: General
7,"Overcast, 33.",Topic 4: General Conditions,0.6,Topic 1: General
8,"Current temp is 36F with light rain falling from cloudy skies and NW winds at 14 gusting to 26 mph. Yesterday's high temp was 66F (3 degrees short of the record of 69F) at 5:48 p.m. Light rain showers and drizzle fell off and on during the morning hours with light to moderate SE winds. By afternoon the showers ended and the winds shifted to light S with overcast skies. During the evening hours light SE winds prevailed with cloudy skies. Overnight, light rain showers began again with moderate NW winds gusting up to 26 mph. Temps dropped sharply during the night from the high 50s down to the mid 30s.",Topic 1: Precipitation,0.44198,Topic 4: Precipitation
9,"Currently mostly cloudy, 30F.",Topic 4: General Conditions,0.732796,Topic 2: Cloudy
